# Домашнее задание 4. Конструирование текстовых признаков из твитов пользователей

## Сбор данных

Первый этап - сбор твитов пользователей. Необходимо подключаться к Twitter API и запрашивать твиты по id пользователя. 
Подключение к API подробно описано в ДЗ 1.

In [1]:
import twitter
CONSUMER_KEY = "t7TY8m7W1PELmEQdD7ai4hnlj"
CONSUMER_SECRET = "TrYFvzvblOaAy9NzEUIwJR4FiIUYjRqigNoNxnSeCQP6ulBX3r"

ACCESS_TOKEN_KEY = "703950096111693824-WXiMMetnK8kataCmP9vXSUYd1sYtQy3"
ACCESS_TOKEN_SECRET = "r0jqDByKuLs9H0cGY0vJvQwYtzpKr49PHd767VFEjRTey"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET)


Populating the interactive namespace from numpy and matplotlib


Для получения твитов пользователя может быть использован метод GetUserTimeline из библиотеки python-twitter. Он позволяет получить не более 200 твитов пользователя.

Метод имеет ограничение по количеству запросов в секунду. Для получения информации о промежутке времени, которое необходимо подождать для повторного обращения к API может быть использован метод `GetSleepTime`. Для получения информации об ограничениях запросов с помощью метода `GetUserTimeLine` необходимо вызывать `GetSleepTime` с параметром "statuses/user_timeline".

Метод GetUserTimeline возвращает объекты типа Status. У этих объектов есть метод AsDict, который позволяет представить твит в виде словаря.

Id пользователей необходимо считать из файла, как было сделано в ДЗ 1.

Необходимо реализовать функцию `get_user_tweets(user_id)`. Входной параметр - id пользователя из файла. Возвращаемое значение - массив твитов пользователя, где каждый твит представлен в виде словаря. Предполагается, что информация о пользователе содержится в твитах, которые пользователь написал сам. Это означает, что можно попробовать отфильтровать ответы другим пользователям, ссылки и ретвиты, а так же картинки и видео, так как наша цель - найти текстовую информацию.

## Разбор текста твита

In [2]:
import time

def get_user_tweets(user_id):
    
    """returns list of tweets as dicts"""
    try: 
        a=api.GetUserTimeline(user_id)
    except:
        sec = api.GetSleepTime('statuses/user_timeline')
        if (sec==0):
            return None
        else:
            print ("Waiting for",sec)
            time.sleep(sec)
          
            return get_user_tweets(user_id)
   
   
    result=[]
    for i in a:
        b=i.AsDict()
        if (not("media" in  b.keys()) and not("retweeted_status" in b.keys() ) and ("text" in b.keys())  ):
            result.append(b["text"])    
    return result 

  

Обработка текста предполагает разбиение текста на отдельные элементы - параграфы, предложения, слова. Мы будем преобразовывать текст твита к словам. Для этого текст необходимо разбить на слова. Сделать это можно, например, с помощью регулярного выражения.

Необходимо реализовать функцию, `get_words(text)`. Входной параметр - строка с текстом. Возвращаемое значение - массив строк (слов). Обратите внимание, что нужно учесть возможное наличие пунктуации и выделить по возможности только слова. 

In [3]:
def get_words(text):
    """returns list of words"""
    return re.findall(r"[\w']+",text)
  


In [52]:
print get_words("Here are different words!")

['Here', 'are', 'different', 'words']


Далее полученные слова необходимо привести к нормальной форме. То есть привести их к форме единственного числа настоящего времени и пр. Сделать это можно с помощью библиотеки nltk. Информацию по загрузке, установке библиотеки и примерах использования можно найти на сайте http://www.nltk.org/

Для загрузки всех необходимых словарей можно воспользоваться методом download из библиотеки nltk.

In [4]:
import nltk
#nltk.download()

Для дальнейшей обработки слова должны быть приведены к нижнему регистру. 

Для приведения к нормальной форме можно использовать `WordNetLemmatizer` из библиотеки nltk. У этого класса есть метод `lemmatize`.

Также необходимо убрать из текста так называемые стоп-слова. Это часто используемые слова, не несущие смысловой нагрузки для наших задач. Сделать это можно с помощью `stopwords` из nltk.corpus

Необходимо реализовать функцию `get_tokens(words)`. Входной параметр - массив слов. Возвращаемое значение - массив токенов.

In [5]:
from nltk.stem.wordnet import WordNetLemmatizer
a=["different","words"]
from nltk.corpus import stopwords
def get_tokens(words):
    i=0
    """returns list of tokens"""
    stop = stopwords.words('english')
    mtzr = WordNetLemmatizer()
    while i<len(words):
        words[i]=words[i].lower()
        words[i]=mtzr.lemmatize(words[i])
        if words[i] in stop:
            words.pop(i)
        else:
            i=i+1
    # your code here
    return words


In [6]:
print get_tokens(["here", "are", "different", "words"])

['different', u'word']


Необходимо реализовать функцию `get_tweet_tokens(tweet)`. Входной параметр - текст твита. Возвращаемое значение -- токены твита. 

In [9]:
def get_tweet_tokens(tweet):
    a=get_words(tweet)
    return get_tokens(a)

def how_many_items_in_list(lst): 
    dct={}
    for i in lst:
        if i in dct:
            dct[i] += 1
        else:
            dct[i] = 1  
    return dct


Необходимо реализовать функцию `collect_users_tokens()`. Функция должна сконструировать матрицу признаков пользователей. В этой матрице строка - пользователь. Столбец - токен. На пересечении - сколько раз токен встречается у пользователя.
Для построения матрицы можно использовать `DictVectorizer` из `sklearn.feature_extraction`.

In [10]:
def collect_users_tokens(x):
    count=0
    result1=[]
    result2=[]
    """returns users list and list of user dicts. Each dict contains frequence of user tokens"""
    for i in x:
        temp=[]
        print count
        
        step1=get_user_tweets(i)
        if step1 is None:
            print "None!"
        else:
            result1.append(i)
            for j in step1:
                temp=temp+get_tweet_tokens(j)
            result2.append(how_many_items_in_list(temp))
            count=count+1
    # your code here
    return result1,result2
STOPWORDS=["co","http","wa","m"]

In [11]:
import pandas as pd
from collections import defaultdict
import json
from sklearn.feature_extraction import DictVectorizer


TRAINING_SET_URL = "Downloads/twitter_train.csv"
df_users = pd.read_csv(TRAINING_SET_URL, sep=",")
x=df_users.values[:,0]
#x=x[:400]
i=0
#for user_id in x:
 #   print i
  #  a=api.GetUserTimeline(user_id)
   # sec = api.GetSleepTime('statuses/user_timeline')
   # if sec!=0:
    #    time.sleep(sec)
    #i=i+1
users, users_tokens = collect_users_tokens(x)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
None!
29
30
31
32
33
None!
33
34
35
36
37
38
39
None!
39
40
41


KeyboardInterrupt: 

In [20]:
list_of_dict=users_tokens
result=""
for i in list_of_dict:
    for j in i.keys():
        temp=""
        temp=j+" "
        result=result+temp*i[j]
print result

roh coach gold enter mastergio uga thanks amp love nwo nwo milk la add randy 4 might might laptop dad get far rake foronly game fan dx dx wedding arriving join ticket one day kl7pq2ftxn 5toz5sr2s5 wpqpjbcar4 southernkrazed zamboni bacon 2vab8vxtxv set vegasseason tea wrestling back wwe wwe wwe wwe terminatorimaxsweeps wretlemania nextbook chef' rt rt leaf jewelry father's rkoheardroundtheworld 500 new sweepstakes dating mancrates amen iridewithelrey mightyleaf free free like giveaway giveaway giveaway julezz6 wrestlemania wrestlemania wrestlemania wrestlemania 'the oktoberfest oktoberfest sister hbk bos audiobook bpjb5iajti i'm i'm luck via via co co co co co co co co co win win win win win win win autographed vip ppv ring trump 12weeksvs top anyone beer survive entered entered entered entered vxs rkos indystar diamond holstedjewelers season season great 99 watchroh white igbnvxlcvu 12 tank hour gotta 5khhqflo2f vega vega foo0cjnnlp v loving pack xdr2fxkb78 aloofbobcat stream tonight t

In [43]:
v = DictVectorizer()
vs = v.fit_transform(users_tokens)
import numpy as np
np.savez("out_4.dat", data=vs, users=users, users_tokens=users_tokens )

Далее для получения представления о полученной информацию о токенах предлагается отобразить ее в виде облака тэгов. [Подсказка](http://anokhin.github.io/img/tag_cloud.png). 

In [ ]:
from scipy.misc import imread
def draw_tag_cloud(result):
    twitter_mask = imread('sphere.png', flatten=True)
    wordcloud = WordCloud(
                      
                      stopwords=STOPWORDS,
                      background_color='white',
                      width=1800,
                      height=1400,
                      mask=twitter_mask
            ).generate(result)

    plt.imshow(wordcloud)
    plt.axis("off")
    plt.savefig('./my_twitter_wordcloud_2.png', dpi=300)
    plt.show()
    return
draw_tag_cloud(result)